In [ ]:
import os
import sys
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import display

sys.path.append(os.getcwd() + "/../")

import bfcnn

In [ ]:
# setup environment
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.compat.v1.enable_eager_execution()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
config = bfcnn.CONFIGS_DICT["resnet_color_1x6_bn_32x32_3x3_256x256_l1_relu"]

models = bfcnn.model_builder(config=config["model_denoise"])

denoiser_model = models.denoiser

In [ ]:
with tf.GradientTape() as g:
    x = \
        tf.random.truncated_normal(
            seed=0,
            mean=0.0,
            stddev=0.25,
            shape=(1, 16, 24, 3))
    x = tf.clip_by_value(x, clip_value_min=-0.5, clip_value_max=0.5)
    g.watch(x)
    y = denoiser_model(x)

jacobian = g.jacobian(y, x)

print(jacobian)